# Tugas 03 : **Melakukan Pengujian dan Deploy**

NAMA : Mohammad Hasan Basri

NIM  : 210411100169

MATA KULIAH : Pencarian dan Penambangan Web - A

In [20]:
from google.colab import drive
drive.mount('/content/drive')
import pickle
import pandas as pd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
df = pd.read_csv("/content/drive/My Drive/PPWA/report/Tugas-PPWA/Hasil_preprocesing.csv")
df.head()

,judul,tanggal,isi,kategori,cleansing,case_folding,tokenize,Filtering/stopword removal
0,Iran Serukan Negara Muslim Bersatu Hentikan Ke...,"Kamis, 17 Okt 2024 10:01 WIB",Jakarta - Presiden Iran Masoud Pezeshkian mene...,Keislaman,Jakarta Presiden Iran Masoud Pezeshkian menek...,jakarta presiden iran masoud pezeshkian menek...,"['jakarta', 'presiden', 'iran', 'masoud', 'pez...",jakarta presiden iran masoud pezeshkian meneka...
1,"Bacaan Al-Qur'an Surah Asy-Syu'ara: Arab, Lati...","Kamis, 17 Okt 2024 09:30 WIB",Jakarta - Surah Asy-Syu'ara adalah surah ke-26...,Keislaman,Jakarta Surah AsySyuara adalah surah ke dalam...,jakarta surah asysyuara adalah surah ke dalam...,"['jakarta', 'surah', 'asysyuara', 'adalah', 's...",jakarta surah asysyuara surah alquran ayat sur...
2,"Rukun Iman Ke-2, Setiap Muslim Wajib Mengimani...","Kamis, 17 Okt 2024 08:45 WIB",Jakarta - Malaikat adalah makhluk ciptaan Alla...,Keislaman,Jakarta Malaikat adalah makhluk ciptaan Allah...,jakarta malaikat adalah makhluk ciptaan allah...,"['jakarta', 'malaikat', 'adalah', 'makhluk', '...",jakarta malaikat makhluk ciptaan allah swt mus...
3,Kalender Ramadhan 2025 Muhammadiyah dan Predik...,"Kamis, 17 Okt 2024 08:00 WIB",Jakarta - Kalender Ramadhan 2025 versi Muhamma...,Keislaman,Jakarta Kalender Ramadhan versi Muhammadiyah...,jakarta kalender ramadhan versi muhammadiyah...,"['jakarta', 'kalender', 'ramadhan', 'versi', '...",jakarta kalender ramadhan versi muhammadiyah t...
4,20 Gambaran Kehidupan di Neraka Menurut Al-Qur...,"Kamis, 17 Okt 2024 07:15 WIB",Jakarta - Gambaran kehidupan di neraka selalu ...,Keislaman,Jakarta Gambaran kehidupan di neraka selalu m...,jakarta gambaran kehidupan di neraka selalu m...,"['jakarta', 'gambaran', 'kehidupan', 'di', 'ne...",jakarta gambaran kehidupan neraka pengingat ke...


**TF-IDF (Term Frequency-Inverse Document Frequency)**


---


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Menginisialisasi TfidfVectorizer
vectorizer = TfidfVectorizer()

# Menghitung TF-IDF
tfidf_matrix = vectorizer.fit_transform(df['Filtering/stopword removal'])

# Mengubah hasilnya menjadi DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
tfidf_df.head(10)
# Menyimpan hasil ke file CSV
tfidf_df.to_csv('hasil_tfidf.csv', index=False)

In [23]:
# Menggunakan kolom 'cleaned_text' sebagai teks input dan 'kategori' sebagai label
texts = df['Filtering/stopword removal'].fillna('')  # Mengganti NaN dengan string kosong jika ada
labels = df['kategori'].fillna('unknown')  # Mengganti NaN di kategori jika ada

# Melihat label yang digunakan untuk klasifikasi
print(labels.unique())

['Keislaman' 'Pariwisata']


**mengubah data teks menjadi representasi numerik menggunakan TF-IDF Vectorizer, kemudian membagi dataset yang sudah diubah ini menjadi data latih dan data uji menggunakan fungsi train_test_split.**

In [24]:
from sklearn.model_selection import train_test_split
# Melakukan transformasi TF-IDF
tfidf = TfidfVectorizer(max_features=None)  # Menggunakan 5000 fitur teratas
X_tfidf = tfidf.fit_transform(texts)

# Memisahkan dataset menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, labels, test_size=0.2, random_state=42)

# Melihat bentuk dari hasil TF-IDF
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)

Shape of X_train: (80, 7517)
Shape of X_test: (20, 7517)


**membuat dan melatih model Logistic Regression dengan regularisasi untuk mencegah overfitting, kemudian menggunakan model tersebut untuk memprediksi hasil pada data uji.**

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Membuat model Logistic Regression dengan regularisasi
model = LogisticRegression(max_iter=1000, C=0.3)  # Mengurangi nilai C untuk menambah regularisasi

# Melatih model dengan data yang sudah diresample (jika diperlukan)
model.fit(X_train, y_train)

# Memprediksi hasil pada data testing
y_pred = model.predict(X_test)

**Evaluasi Hasil**

---


In [26]:
# Evaluasi hasil prediksi
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Menampilkan laporan klasifikasi
print(classification_report(y_test, y_pred))

Accuracy: 75.00%
              precision    recall  f1-score   support

   Keislaman       1.00      0.58      0.74        12
  Pariwisata       0.62      1.00      0.76         8

    accuracy                           0.75        20
   macro avg       0.81      0.79      0.75        20
weighted avg       0.85      0.75      0.75        20



**Menyimpan model dan vectorizer (TF-IDF dan Logistic Regression) ke dalam file menggunakan pickle**

---


In [27]:
# Menyimpan tfidf_matrix dan vectorizer menggunakan pickle
with open('tfidf_vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)
# Menyimpan model ke dalam file pickle
with open('logistic_regression_model.pkl', 'wb') as file:
    pickle.dump(model, file)

**PENGUJIAN**

---



In [28]:
import joblib

# Memuat vectorizer dan model
tfidf = joblib.load('tfidf_vectorizer.pkl')
model = joblib.load('logistic_regression_model.pkl')

In [29]:
# Teks baru yang ingin diprediksi
new_text = ['''Jakarta, 25 September 2024 – Pertumbuhan sektor keuangan di Indonesia terus menunjukkan tren positif, terutama di tengah pemulihan ekonomi pasca-pandemi. Data dari Bank Indonesia (BI) menunjukkan peningkatan tajam dalam penggunaan layanan keuangan digital sepanjang tahun 2024, dengan transaksi e-wallet naik 35% dibandingkan periode yang sama tahun lalu.''']

# Preprocessing dan transformasi menggunakan TF-IDF yang sudah disimpan
new_text_tfidf = tfidf.transform(new_text)

# Melakukan prediksi menggunakan model Logistic Regression
prediction = model.predict(new_text_tfidf)

# Hasil prediksi
predicted_category = "Keislaman" if prediction[0] == 'Keislaman' else "Pariwisata"
print(f"Prediksi: {predicted_category}")

Prediksi: Pariwisata


In [30]:
# Mengecek versi scikit-learn
import sklearn
print(sklearn.__version__)

1.5.2
